In [7]:
import cohere


In [8]:
class QueryProcessor:
    def __init__(self, co_api_key):
        # Initialize Cohere Client
        self.cohere_client = cohere.Client(co_api_key)
        
        # Define the prompts
        self.Query_Prompt = """Users aren't always the best at articulating what they're looking for. Your task is to understand the 
        essence of the user query and generate {num_queries} alternate queries to expand the user's query so it's more robust. This way the user will
        receive the most relevant information.

        Examples are delimited by triple backticks (```) below

        ````
        User Query: How can I find the positive in situations that seem negative?

        Alternate Queries:

        1. How can I cultivate optimism and positive thinking in my daily life?
        2. Is it possible to find meaning and purpose in challenging or difficult times?
        3. What are some effective strategies for reframing negative thoughts into positive ones?
        ````

        User Query : How can I develop a mindset that helps me focus on the positive in tough situations?
        ```
        1. What are some techniques to shift my perspective from negativity to positivity?
        2. How do I overcome negative thinking and embrace a more optimistic outlook?
        3. What are the best ways to stay positive when facing adversity or challenges?
        ```

        Generate {num_queries} alternate queries, one on each line, for the following user query:
        --------------------
        User Query: {user_query}
        --------------------

        Alternate Queries:
        """

        self.Hypothetical_Answer_Prompt = """Based on the query provided, generate a plausible, general answer that encapsulates how AI might be used to address this situation.
        The answer should reflect general knowledge and should not be the final answer, but just an informative and plausible response.

        User Query: {user_query}

        Hypothetical Answer:
        """
        
    def generate_queries(self, query, num_queries=4):
        # Generate alternate queries based on the user input
        prompt = self.Query_Prompt.format(user_query=query, num_queries=num_queries)
        
        response = self.cohere_client.generate(
            model='command-xlarge',
            prompt=prompt,
            max_tokens=100,
            temperature=0.7,
            k=1,
            num_generations=1
        )

        generated_text = response.generations[0].text.strip()
        clean_text = generated_text.split('\n')
        clean_queries = [line.strip() for line in clean_text if line.strip() and line.startswith(('1.', '2.', '3.', '4.'))]
        return clean_queries

    def generate_hypothetical_answer(self, query):
        # Generate a plausible hypothetical answer based on the query
        prompt = self.Hypothetical_Answer_Prompt.format(user_query=query)
        
        response = self.cohere_client.generate(
            model='command-xlarge',
            prompt=prompt,
            max_tokens=150,
            temperature=0.7,
            num_generations=1
        )

        hypothetical_answer = response.generations[0].text.strip()
        return hypothetical_answer
